# Work with different data formats (binary, netCDF, .mat)

Nikolay Koldunov

koldunovn@gmail.com

This is part of [**Python for Geosciences**](https://github.com/koldunovn/python_for_geosciences) notes.

================

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np

## Binary data

### Open binary

Data from [Sea Ice Concentrations from Nimbus-7 SMMR and DMSP SSM/I-SSMIS Passive Microwave Data](http://nsidc.org/data/nsidc-0051.html).

In [ ]:
!wget ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/monthly/nt_200709_f17_v01_n.bin

Create file id:

In [ ]:
ice = np.fromfile('nt_200709_f17_v01_n.bin', dtype='uint8')

We use *uint8* data type. 
[List of numpy data types](http://docs.scipy.org/doc/numpy/user/basics.types.html)

The file format consists of a 300-byte descriptive header followed by a two-dimensional array.

In [ ]:
ice = ice[300:]

Reshape

In [ ]:
ice = ice.reshape(448,304)

Simple visualisation of array with imshow (Matplotlib function):

In [ ]:
plt.imshow(ice)
plt.colorbar();

To convert to the fractional parameter range of 0.0 to 1.0, divide the scaled data in the file by 250.

In [ ]:
ice = ice/250.
plt.imshow(ice)
plt.colorbar();

Let's mask all land and missing values:

In [ ]:
ice_masked = np.ma.masked_greater(ice, 1.0)
plt.imshow(ice_masked)
plt.colorbar();

Masking in this case is similar to using *NaN* in Matlab. [More about NumPy masked arrays](http://docs.scipy.org/doc/numpy/reference/maskedarray.html)

### Save binary

In [ ]:
fid = open('My_ice_2007.bin', 'wb')
ice.tofile(fid)
fid.close()

In order to work with other data formats we need to use one of the SciPy submodules:

## SciPy

<img  height="100" src="files/scipy.png" >

General purpose scientific library (that consist of bunch of sublibraries) and builds on NumPy arrays.

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

We are going to use only scipy.io library.

## scipy.io

### Open .mat files

First we have to load function that works with Matlab files:

In [ ]:
from scipy.io import loadmat

We are going to download [Polar science center Hydrographic Climatology (PHC)](http://psc.apl.washington.edu/nonwp_projects/PHC/Climatology.html) for January in Matlab format.

In [ ]:
!wget https://www.dropbox.com/s/0kuzvz03gw6d393/PHC_jan.mat

Open file:

In [ ]:
all_variables = loadmat('PHC_jan.mat')

We can look at the names of variables stored in the file:

In [ ]:
all_variables.keys()

We need only *PTEMP1* (3d potential temperature).

In [ ]:
temp = np.array(all_variables['PTEMP1'])

Check variable's shape:

In [ ]:
temp.shape

Show surface level:

In [ ]:
plt.imshow(temp[0,:,:])
plt.colorbar();

### Open netCDF files

Scipy have function for working with netCDF files, and you can import it with:
    from scipy.io import netcdf
However it only supports netCDF3 format. It is better to use [python netcdf4](https://github.com/Unidata/netcdf4-python) module that have a lot of nice functionality. Moreover NCEP reanalysis data, that we are going to work with are in netCDF4 format.

Import nessesary function:

In [ ]:
from netCDF4 import Dataset

I am going to download [NCEP reanalysis data](http://www.esrl.noaa.gov/psd/data/reanalysis/reanalysis.shtml). Surface 4 daily air temperature for 2012.

In [ ]:
!wget ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis/surface/air.sig995.2012.nc

#Alternative for the times of US goverment shutdowns:
#!wget http://database.rish.kyoto-u.ac.jp/arch/ncep/data/ncep.reanalysis/surface/air.sig995.2012.nc

Create file id:

In [ ]:
fnc = Dataset('air.sig995.2012.nc')

It's not really file id, it's *netcdf_file* object, that have some methods and attributes:

In [ ]:
fnc.description

In [ ]:
fnc.history

list variables

In [ ]:
fnc.variables

Access information about variables

In [ ]:
air = fnc.variables['air']

This time we create *netcdf_variable* object, that contain among other things attributes of the netCDF variable as well as data themselves. 

In [ ]:
air.actual_range

In [ ]:
air.long_name

In [ ]:
air.units

In [ ]:
air.shape

We can access the data by simply using array syntax. Here we show first time step of our data set:

In [ ]:
plt.imshow(air[0,:,:])
plt.colorbar();

### Save netCDF file

Minimalistic variant :)

In [ ]:
!rm test_netcdf.nc
fw = Dataset('test_netcdf.nc', 'w')

fw.createDimension('t', 1464)
fw.createDimension('y', 73)
fw.createDimension('x', 144)

air_var = fw.createVariable( 'air','float32', ('t', 'y', 'x'))
air_var[:] = air[:]
fw.close()

More descriptive variant:

In [ ]:
!rm test_netcdf.nc
fw = Dataset('test_netcdf.nc', 'w')

fw.createDimension('TIME', 1464)
fw.createDimension('LATITUDE', 73)
fw.createDimension('LONGITUDE', 144)

time = fw.createVariable('TIME', 'f', ('TIME',))
time[:] = fnc.variables['time'][:]
time.units = 'hours since 1-1-1 00:00:0.0' 

lat  = fw.createVariable('LATITUDE', 'f', ('LATITUDE',))
lat[:] = fnc.variables['lat'][:]

lon = fw.createVariable('LONGITUDE', 'f', ('LONGITUDE',))
lon[:] = fnc.variables['lon'][:]

ha = fw.createVariable('New_air','f', ('TIME', 'LATITUDE', 'LONGITUDE'))
ha[:] = air[:]
ha.missing_value = -9999.

fw.close()

## Links

* [PyNio - library that reads NetCDF, GRIB, HDF and so on ... ](http://www.pyngl.ucar.edu/Nio.shtml)
* [netCDF4 - advanced netCDF module](http://netcdf4-python.googlecode.com/svn/trunk/docs/netCDF4-module.html)
* [Scipy cookbook](http://wiki.scipy.org/Cookbook)